## Creating folders for each WARENGRUPPE along with the product data for that WR

This is a one-time used code to structure the data so that products for each product group are in their separate folder along with their prompt format and allowed keys as well as their output csv and json files. This will structure the data so that each product group is treated individually based on its attributes. 

In [1]:
## Importing libraries and initializing values class to import the relevant variables
import pandas as pd
from values import *
import os

val = Values()
encoding_ = 'cp850'
encoding_errors='replace'

In [2]:
is_text = pd.read_csv("/Volumes/MARAL/CSV/F01/V4AR1005.csv", sep=";", encoding="cp850")
mar = pd.read_csv("/Volumes/MARAL/CSV/F01/V2AR1001.csv", sep=";", encoding="cp850",usecols=['NUMMER','BANUMMER','WARENGR','TEXT_KZ','WM'])
mar = mar.rename(columns={'NUMMER':'STAMMNUMMER'})
mar = mar.rename(columns={'BANUMMER':'NUMMER','WARENGR':'WG_CODE'})
zuz = pd.read_csv("/Volumes/MARAL/CSV/F01/V4AR1009.csv", sep=";", encoding="cp850",usecols=['NUMMER','VAR_TEXT'])
wg = pd.read_excel(val.wr_filepath)
wg['WG_CODE'] = wg['WG_CODE'].astype(str).str.replace('.0','').str.strip()
mar['WG_CODE'] = mar['WG_CODE'].astype(str).str.replace('.0','').str.strip()

KeyboardInterrupt: 

In [ ]:
mar_zuz = pd.merge(mar,zuz,how='left',on='NUMMER')
mar_zuz_text = mar_zuz.merge(
    is_text, left_on="TEXT_KZ", right_on="TEXTNR", how="left"
)
mar_zuz_text_wg = mar_zuz_text.merge(wg,how='left',on='WG_CODE')    
df = mar_zuz_text_wg[(~mar_zuz_text_wg["NUMMER"].str.startswith("R", na=False)) & (mar_zuz_text_wg["WM"].isna()) & (~mar_zuz_text_wg["STAMMNUMMER"].str.endswith("P")) ]


In [ ]:
df = df[['STAMMNUMMER','WM',  'NUMMER', 'INTERNET', 'KATALOG', 'MAILING', 'BANAME',  'TEXT9', 'TEXT10', 'TEXT11', 'WG_NAME','VAR_TEXT','WG_CODE']]
df = df.fillna("")
df['BESCHREIBUNG'] = df['BANAME'] + "|" + df['VAR_TEXT'] + "|" + df['INTERNET']
df = df[df['BESCHREIBUNG'] != r"\s*"]
df = df[['STAMMNUMMER','NUMMER','BANAME','WG_NAME','BESCHREIBUNG','WG_CODE']]

### Data Processing

### Data Structure Creation

In the following cell, different tables are connected together to create a list of all the artikels that are in the shop, and that have a description, so that we can extract their attributes from their description and VAR_TEXT. some lines are commented, because for example the katalog data is no longer needed for this purpose. But can be added if needed in the future.
Also note that the folder name is now changed to Product_mining_2 instead of Product_mining. So we have two folders, and the old one is a bit processed and can be used for comparisons.

In [ ]:
wg_data = []
for num, vals in zip(wg['WG_CODE'],wg['WG_NAME']):
    ## iterating over warengruppe
    artikels = df[df['WG_CODE'] == str(num)]
    ## connecting marketing artikels for each warnegruppe with the shop list
    
    ## data cleaning (removing html codes and unnecessary characters)
    artikels['BESCHREIBUNG'] = artikels['BESCHREIBUNG'].str.replace(' ',' ',regex=True)
    artikels['BESCHREIBUNG'] = artikels['BESCHREIBUNG'].str.replace(r'(<[^>]*>)',' ',regex=True,case=False)
    artikels['BESCHREIBUNG'] = artikels['BESCHREIBUNG'].str.replace(r'&nbsp_|&nbsp;|&Oslash',' ',regex=True)
    # artikels = artikels[artikels['BESCHREIBUNG'].isna() == False]
    ## preparing the product description to have all the information by concatenating both names and description together
    # artikels['BESCHREIBUNG'] = artikels['NAME'] + ' - ' + artikels['Var_Text'] + ' - ' + artikels['NET_BESCHREIBUNG'] 
    # artikels = artikels[artikels['WARENGR'].isna() == False]
    # artikels = artikels.drop_duplicates('NUMBER')
    # artikels = artikels.dropna(subset='NUMBER')
    artikels = artikels[['STAMMNUMMER','NUMMER','WG_CODE','WG_NAME','BANAME','BESCHREIBUNG']]
    wg_data.append([num,vals,len(artikels)])

    if len(artikels) != 0:
        new_val = vals.replace('/','_')
        paths = os.path.join(val.parent_dir, str(num)+'_'+str(new_val))
        print(str(num) + ' ' + str(len(artikels)))
        # Building folders based on the warengruppe numbers
        if not os.path.exists(paths):
            os.mkdir(paths) 
        artikels.to_excel(f'{paths}/{num}_{new_val}.xlsx', engine='xlsxwriter',index=False)
        
wg_data

/var/folders/5j/sk834jzs3cg8kh3yp7743_ym0000gp/T/ipykernel_55269/2281282486.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artikels['BESCHREIBUNG'] = artikels['BESCHREIBUNG'].str.replace(' ',' ',regex=True)
/var/folders/5j/sk834jzs3cg8kh3yp7743_ym0000gp/T/ipykernel_55269/2281282486.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artikels['BESCHREIBUNG'] = artikels['BESCHREIBUNG'].str.replace(r'(<[^>]*>)',' ',regex=True,case=False)
/var/folders/5j/sk834jzs3cg8kh3yp7743_ym0000gp/T/ipykernel_55269/228

1001 21
1002 40
1003 75
1004 175
1005 89
1006 35
1007 91
1008 95
1009 11
1010 46
1011 36
1012 237
1013 118
1014 47
1015 114
1016 13
1017 8
1018 59
1019 9
1020 36
1021 35


/var/folders/5j/sk834jzs3cg8kh3yp7743_ym0000gp/T/ipykernel_55269/2281282486.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artikels['BESCHREIBUNG'] = artikels['BESCHREIBUNG'].str.replace(' ',' ',regex=True)
/var/folders/5j/sk834jzs3cg8kh3yp7743_ym0000gp/T/ipykernel_55269/2281282486.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artikels['BESCHREIBUNG'] = artikels['BESCHREIBUNG'].str.replace(r'(<[^>]*>)',' ',regex=True,case=False)
/var/folders/5j/sk834jzs3cg8kh3yp7743_ym0000gp/T/ipykernel_55269/228

1022 41
1023 9
1024 3
1025 104
1026 138
1027 47
1028 284
1029 14
1030 56
1031 28
1032 113
1033 135
1034 206
1035 85
1036 123
1037 19
1038 93
1101 64
1102 9


/var/folders/5j/sk834jzs3cg8kh3yp7743_ym0000gp/T/ipykernel_55269/2281282486.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artikels['BESCHREIBUNG'] = artikels['BESCHREIBUNG'].str.replace(' ',' ',regex=True)
/var/folders/5j/sk834jzs3cg8kh3yp7743_ym0000gp/T/ipykernel_55269/2281282486.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artikels['BESCHREIBUNG'] = artikels['BESCHREIBUNG'].str.replace(r'(<[^>]*>)',' ',regex=True,case=False)
/var/folders/5j/sk834jzs3cg8kh3yp7743_ym0000gp/T/ipykernel_55269/228

...

/var/folders/5j/sk834jzs3cg8kh3yp7743_ym0000gp/T/ipykernel_55269/2281282486.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artikels['BESCHREIBUNG'] = artikels['BESCHREIBUNG'].str.replace(' ',' ',regex=True)
/var/folders/5j/sk834jzs3cg8kh3yp7743_ym0000gp/T/ipykernel_55269/2281282486.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artikels['BESCHREIBUNG'] = artikels['BESCHREIBUNG'].str.replace(r'(<[^>]*>)',' ',regex=True,case=False)
/var/folders/5j/sk834jzs3cg8kh3yp7743_ym0000gp/T/ipykernel_55269/228

2421 128
2422 45
2423 89
2424 16
2425 114
2426 56
2427 46
2428 334
2429 461
2430 67
9000 11
9101 977


/var/folders/5j/sk834jzs3cg8kh3yp7743_ym0000gp/T/ipykernel_55269/2281282486.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artikels['BESCHREIBUNG'] = artikels['BESCHREIBUNG'].str.replace(' ',' ',regex=True)
/var/folders/5j/sk834jzs3cg8kh3yp7743_ym0000gp/T/ipykernel_55269/2281282486.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artikels['BESCHREIBUNG'] = artikels['BESCHREIBUNG'].str.replace(r'(<[^>]*>)',' ',regex=True,case=False)
/var/folders/5j/sk834jzs3cg8kh3yp7743_ym0000gp/T/ipykernel_55269/228

9102 11
9103 53
9104 13
9105 86
9106 839
9107 592
9108 217
9109 168
9200 606
9900 34405


[['1001', 'Öfen - Mikrowellen', 21],
 ['1002', 'Fritteusen - Zubehör', 40],
 ['1003', 'Herd - Kochfelder ', 75],
 ['1004', 'Kaffeegeräte', 175],
 ['1005', 'Wassergeräte - Teegeräte', 89],
 ['1006', 'Mixer - Zubehör', 35],
 ['1007', 'Stabmixer - Pürierer - Zubehör', 91],
 ['1008', 'Mühlen - Zubehör', 95],
 ['1009', 'Waagen', 11],
 ['1010', 'Waffeleisen - Kuchenbäcker', 46],
 ['1011', 'Kochautomat', 36],
 ['1012', 'Küchenmaschinen', 237],
 ['1013', 'Küchenmaschinenzubehör', 118],
 ['1014', 'Klimageräte - Heizer', 47],
 ['1015', 'Küchenspezialgeräte - Zubehör', 114],
 ['1016', 'Haushaltsspezialgeräte', 13],
 ['1017', 'Uhren', 8],
 ['1018', 'Entsafter - Pressen', 59],
 ['1019', 'Brotbäcker ', 9],
 ['1020', 'Eisgeräte - Zubehör', 36],
 ['1021', 'Pastageräte - Zubehör ', 35],
 ['1022', 'Sous-vide Vakuumierer Zubehör', 41],
 ['1023', 'Leitern', 9],
 ['1024', 'Mülleimer', 3],
 ['1025', 'Messer', 104],
 ['1026', 'Schneiden - Reiben - Hobeln', 138],
 ['1027', 'Schneidebretter', 47],
 ['1028', 'K

## Code for creating the Product_mining lists based on Marketing Artikel Bezeichnung data BANAM1,2, and 3

In [ ]:
wg_data = []
for num, vals in zip(warengr['WG_CODE'],warengr['WG_NAME']):
    ## iterating over warengruppe
    artikels_in_wr = marketing_art[marketing_art['WARENGR'] == str(num)]
    ## connecting marketing artikels for each warnegruppe with the shop list
    # artikels = pd.merge(artikels_in_wr,shop,how='left',left_on="NUMBER",right_on="StoreId")
    artikels = pd.merge(artikels_in_wr,mark_bez_df,how='left',left_on='NUMBER',right_on='NUMMER')
    # artikels = artikels.dropna(subset='WARENGR')

    artikels = pd.merge(artikels,shop,how='right',left_on="NUMBER",right_on="StoreId")


    artikels = artikels.rename(columns={'NUMMER_x':'NUMMER'})
    artikels = artikels.drop_duplicates('NUMMER')
    artikels = artikels.dropna(subset='NUMMER')
    artikels = artikels[['WM','NUMMER','GROESSE','FARBE','WARENGR','BANAME1','BANAME2','BANAME3']]
    wg_data.append([num,vals,len(artikels)])

    if len(artikels) != 0:
        new_val = vals.replace('/','_')
        paths = os.path.join(val.parent_dir, str(num)+'_'+str(new_val))
        print(paths)
        # print(path)
        # Building folders based on the warengruppe numbers
        if not os.path.exists(paths):
            os.mkdir(paths) 
        # artikels.to_csv(f'{paths}/{num}_{new_val}.csv',encoding='utf-8', sep=',',index=False)
        # Apply the cleaning function to all string elements in the DataFrame
        # artikels = artikels.apply(convert_encoding)
        # artikels.to_csv(f'{paths}/{num}_{new_val}.csv',index=False,sep=';',encoding='utf-8')
        # artikels.to_excel(f'{paths}/{num}_{new_val}.xlsx',index=False, engine='openpyxl')
        artikels.to_excel(f'{paths}/{num}_{new_val}_shop.xlsx', engine='xlsxwriter',index=False)

        # artikels.to_json(f'{paths}/{num}_{new_val}.json',index=False)
    # else:
    #     paths = os.path.join(val.parent_dir, str(num))
    #     for file in os.listdir(paths):
    #         os.remove(os.path.join(paths,file))
    #     print(paths)
    #     os.rmdir(paths)

        ## inserting the csv file of the products for each warengruppe and adding a prompt placeholder text (to be filled)
        # with open(f'{path}/prompt_{num}.txt','w') as txt:
        #     txt.write("prompt-to-be-filled")
        
wg_data

### Saving WG_artikel_3 which is list of WGs

In [ ]:
# wg_df = pd.DataFrame(wg_data,columns=['WG_ID','WG_NAME','ITEMS_IN_WG'])
# # wg_df.to_csv('/Users/maralsheikhzadeh/Documents/Codes/useful-exports/WG_artikels_3.csv',sep=';',index=False,encoding='utf-8')
# wg_df.to_excel('/Users/maralsheikhzadeh/Documents/Codes/useful-exports/WG_artikels_3.xlsx',index=False, engine='openpyxl')